# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.3585,55.7679,295.90,78,59,8.73,RE,1621471452
1,Kutum,14.2000,24.6667,296.74,38,0,1.90,SD,1621471452
2,Kirakira,-10.4544,161.9205,300.22,86,100,3.36,SB,1621471153
3,Port Hedland,-20.3167,118.5667,293.18,56,0,2.57,AU,1621471176
4,Pisco,-13.7000,-76.2167,292.60,82,0,5.14,PE,1621471134


In [4]:
weather_data.keys()

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#configuring gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Lng"]].astype(float)

weight = weather_data["Humidity"].astype(float)


In [7]:
# Plot Heatmap
fig = gmaps.figure()

# Create a humidity Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights= weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.3585,55.7679,295.90,78,59,8.73,RE,1621471452
1,Kutum,14.2000,24.6667,296.74,38,0,1.90,SD,1621471452
2,Kirakira,-10.4544,161.9205,300.22,86,100,3.36,SB,1621471153
3,Port Hedland,-20.3167,118.5667,293.18,56,0,2.57,AU,1621471176
4,Pisco,-13.7000,-76.2167,292.60,82,0,5.14,PE,1621471134
...,...,...,...,...,...,...,...,...,...
366,Sonkovo,57.7782,37.1571,287.97,96,100,4.57,RU,1621471572
367,Honningsvåg,70.9821,25.9704,277.84,92,100,3.74,NO,1621471573
368,Kingston,17.9970,-76.7936,301.68,81,20,8.75,JM,1621471524
369,Katobu,-4.9408,122.5276,302.03,71,24,2.95,ID,1621471243


In [22]:
# Filtering data by ideal conditions
weather_df = weather_data[(weather_data['Wind Speed'] < 5) & (weather_data['Cloudiness'] < 10)]
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Kutum,14.2000,24.6667,296.74,38,0,1.90,SD,1621471452
3,Port Hedland,-20.3167,118.5667,293.18,56,0,2.57,AU,1621471176
19,Busselton,-33.6500,115.3333,289.11,71,6,2.12,AU,1621471119
23,Puerto Ayora,-0.7393,-90.3518,296.79,89,0,2.68,EC,1621471120
28,Cabo San Lucas,22.8909,-109.9124,305.10,61,1,1.54,MX,1621471151


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
# blank column for hotel name
hotel_df = weather_df.copy()
hotel_df["Hotel Name"]= ""

In [25]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Kutum,14.2000,24.6667,296.74,38,0,1.90,SD,1621471452,
3,Port Hedland,-20.3167,118.5667,293.18,56,0,2.57,AU,1621471176,
19,Busselton,-33.6500,115.3333,289.11,71,6,2.12,AU,1621471119,
23,Puerto Ayora,-0.7393,-90.3518,296.79,89,0,2.68,EC,1621471120,
28,Cabo San Lucas,22.8909,-109.9124,305.10,61,1,1.54,MX,1621471151,


In [34]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [35]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Kutum,14.2000,24.6667,296.74,38,0,1.90,SD,1621471452,Dibbo areaمنطقة دبو
3,Port Hedland,-20.3167,118.5667,293.18,56,0,2.57,AU,1621471176,The Esplanade Hotel
19,Busselton,-33.6500,115.3333,289.11,71,6,2.12,AU,1621471119,Observatory Guest House
23,Puerto Ayora,-0.7393,-90.3518,296.79,89,0,2.68,EC,1621471120,Finch Bay Galapagos Hotel
28,Cabo San Lucas,22.8909,-109.9124,305.10,61,1,1.54,MX,1621471151,Tesoro Los Cabos
31,Port Elizabeth,-33.9180,25.5701,289.31,94,0,2.57,ZA,1621471234,39 On Nile Guest House
37,Qeshm,26.9581,56.2719,302.15,69,4,2.06,IR,1621471352,Qeshm Kavan Hotel
40,Upernavik,72.7868,-56.1549,270.44,79,2,4.00,GL,1621471171,Gina's Guesthouse - Upernavik
48,Walvis Bay,-22.9575,14.5053,287.64,80,0,1.83,NaN,1621471305,Oyster Box Guesthouse
49,Kiryat Gat,31.6100,34.7642,293.71,100,0,0.45,IL,1621471323,OlusHome


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
type(hotel_info)

list

In [40]:
#configuring gmaps
gmaps.configure(api_key=g_key)

In [42]:
# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lat', 'Lng']]

# Create a marker_layer using the hotel list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content= hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))